In [6]:
# Quick file to guide the creation of the prediction algorithm on the live website

import pickle
import requests
import numpy as np

regional_good_venues = pickle.load( open( "SF/regional_good_venues.pickle", "rb" ) )
master_genre_map = pickle.load( open( "SF/master_genre_map.pickle", "rb" ) )
master_genre_idf = pickle.load( open( "SF/master_genre_idf.pickle", "rb" ) )
master_venue_vect = pickle.load( open( "SF/master_venue_vect.pickle", "rb" ) )
master_venue_list = pickle.load( open( "SF/master_venue_list.pickle", "rb" ) )
venue_ids = pickle.load( open( "SF/venue_ids.pickle", "rb" ) )
venue_list = pickle.load( open( "SF/venue_list.pickle", "rb" ) )
region_artists = pickle.load( open( "SF/regional_subset.pickle", "rb" ) )
locations = pickle.load( open( "SF/locations.pickle", "rb" ) )
k_means = pickle.load( open( "SF/k_means.pickle", "rb" ) )

# Fetch index for given entry of list
def getIndex(element, my_list):
    # Create a generator
    gen = (i for i,x in enumerate(my_list) if x == element)
    for i in gen:
        return i

# Order-1 normalization (a1+a2+...=1)
def norm1(v):
    scaled_vector = []
    total_A = float(sum(v))
    if total_A == 0:
        return None
    for i in v:
        scaled_vector.append(i/total_A)
    return scaled_vector


token = 'WL4JBSOI4ZGXPY8FN'
raw_string = "weezer"
artists = raw_string.split(",")

query_params = { 'api_key': token,
                 'name': "",
                 'bucket': 'familiarity',
                 'bucket': 'terms'}

endpoint = 'http://developer.echonest.com/api/v4/artist/profile?'

# One pack per artist
venue_pack = []
for artist in artists: #artists.split(","):
    
    query_params['name'] = artist.lstrip().rstrip()
    
    response = requests.get(endpoint, params=query_params)
    full_data = response.json()

    if response.json()['response']['status']['message'] != 'Success':
        #return render_template('index.html', form1=form1, message="", coords=json.dumps(locations))
        print "No artist found for %s!" % artist
        
    terms = full_data['response']['artist']['terms']
    
    genre_vec = [0] * len(master_genre_map)
    # Loop over genres, make vector of frequencies
    for term in terms:
        # Get index of genre in master_genre_map
        indx = getIndex(term['name'], master_genre_map)
        if indx is None:
            continue
        # Set corresponding vector entry to value of frequency
        genre_vec[indx] = term['frequency']
    if norm1(genre_vec) is None:
        continue
    # Set key to artist name, value is array of (normalized) freqs
    scaled_vec = np.array(norm1(genre_vec))
    # Apply tf-idf
    tfidf = scaled_vec*master_genre_idf
    # Last step, normalize vector:
    p_index = k_means.predict(tfidf/np.linalg.norm(tfidf))
    
    venues = []
    for i in range(len(master_venue_list)):
        if k_means.labels_[i] == p_index[0]:
            target_venue = dict()
            target_venue['name'] = master_venue_list[i]
            ind = getIndex(master_venue_list[i], venue_list)
            target_venue['lat'] = locations[ind][0]
            target_venue['lon'] = locations[ind][1]
            target_venue['vid'] = venue_ids[ind]
            venues.append(target_venue)

    venue_pack.append(venues)
# Predict cluster
print venue_pack


[[{'lat': u'37.8286965', 'lon': u'-122.2606041', 'name': u'1-2-3-4 Go! Records', 'vid': 955226}, {'lat': u'37.7842566', 'lon': u'-122.4332961', 'name': u'The Fillmore', 'vid': 6239}, {'lat': u'37.66169', 'lon': u'-121.887367', 'name': u'Alameda County Fairgrounds', 'vid': 29213}, {'lat': u'37.8940799', 'lon': u'-122.0627288', 'name': u'Red House', 'vid': 46434}]]


In [8]:
# API call to get event information:
# API token for Songkick
# LA=17835, SF=26330, Chicago=9426, SD=11086
import requests
#import pickle


# Return [ name, date, [artist ids] ]
# date of first event = getArtists(vid)[0]
# List of artists that night = getArtists(vid)[1]
def getArtists(venue_id):
    token = 'WGLTwENb36bAgHNc'
    query_params = { 'apikey': token,
                     'per_page': 50,
                     'page': 1}
    endpoint = 'http://api.songkick.com/api/3.0/venues/' + venue_id + '/calendar.json?'

    response = requests.get(endpoint, params=query_params)
    total_results = response.json()['resultsPage']['totalEntries']
    if total_results == 0:
        return []
    
    full_data = response.json()['resultsPage']['results']['event']
    if not full_data:
        return []
    
    ev_date = full_data[0]['start']['date']
    ev_name = full_data[0]['displayName']
    print ev_name
    ev_artists = []
    # 0 means first event, could change it to future events
    for artist in full_data[0]['performance']:
        ev_artists.append(artist['artist']['displayName'])
        #print artist['artist']['displayName'], artist['artist']['id']

    return [ev_name, ev_date, ev_artists]


def getEvtVector(artist_names):
    ev_vector = np.array([0.] * len(master_genre_idf))
    for artist in artist_names:
        if artist in region_artists:
            ev_vector += region_artists[artist]
    return ev_vector


print getEvtVector(getArtists('5832')[2])


#for key, val in full_data[0].iteritems():
#    print key, "::::\n", val


Outside Lands Music & Arts Festival 2015
[ 0.45110003  3.78623492  2.36451641 ...,  0.          0.          0.        ]
